In [3]:
!pip install transformers datasets evaluate rouge_score accelerate

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1fc60e2a3588b812f96c1128a73aa3df1f2b16d2caadc07788bf0cdfad2f502b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
import transformers
transformers.__version__

'4.44.2'

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='openai/MMMLU', author='openai', sha='6f08ae4f7e18264d1bd7e58e889376c5a58e3e2e', created_at=datetime.datetime(2024, 9, 13, 16, 37, 19, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 9, 23, 16, 54, 48, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=3096, downloads_all_time=None, likes=354, paperswithcode_id=None, tags=['task_categories:question-answering', 'language:ar', 'language:bn', 'language:de', 'language:es', 'language:fr', 'language:hi', 'language:id', 'language:it', 'language:ja', 'language:ko', 'language:pt', 'language:sw', 'language:yo', 'language:zh', 'license:mit', 'size_categories:100K<n<1M', 'format:csv', 'modality:text', 'library:datasets', 'library:dask', 'library:mlcroissant', 'library:polars', 'arxiv:2009.03300', 'region:us'], card_data=None, siblings=None)
DatasetInfo(id='google/frames-benchmark', author='google', sha='4f4c666ca0ca6997a1664e10217152d298e6d3c9', created_at=datetime.datetime(2024

In [9]:
from datasets import load_dataset

billsum_ds = load_dataset("FiscalNote/billsum")
billsum_ds

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [50]:
billsum_ds = load_dataset("FiscalNote/billsum", split="train[:1%]")
billsum_ds

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 189
})

In [11]:
billsum_ds=billsum_ds.train_test_split(test_size=0.2, seed=42)
billsum_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 454
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 114
    })
})

In [12]:
def clean_txt(example):
    for txt in ["summary", "title"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example

In [13]:
cleaned_billsum_ds = billsum_ds.map(clean_txt)
cleaned_billsum_ds

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 454
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 114
    })
})

In [14]:
cleaned_billsum_ds['train']["summary"][7]

'rio grande del norte national conservation area establishment act - establishes the rio grande del norte national conservation area in new mexico, consisting of approximately 235,980 acres of public land in taos and rio arriba counties.requires the secretary of the interior to ensure the protection of religious and cultural sites in the conservation area and to provide access to them by tribal members for traditional cultural and customary uses.requires the secretary to develop a management plan for the conservation area.bars the changing of the management status of any area within the boundary of the conservation area that is: (1) designated as a component of the national wild and scenic rivers system; or (2) managed as an area of critical environmental concern.designates the cerro del yuta wilderness and rio san antonio wilderness as wilderness and as components of the national wilderness preservation system.releases the public land within the san antonio wilderness study area not d

In [15]:
 billsum_ds['train']["summary"][0]

'Kids Invest and Develop Savings Act of 2007 - Amends the Internal Revenue Code to: (1) increase the allowable amount of the tax credit for retirement savings; (2) expand eligibility for such credit to taxpayers with adjusted gross incomes up to $95,000 ($150,000 for joint returns); (3) establish tax-exempt Roth individual retirement accounts (IRAs) for individuals under the age of 25; and (4) allow a tax credit for contributions to such IRAs.'

In [16]:
 cleaned_billsum_ds['train']["summary"][0]

'kids invest and develop savings act of 2007 - amends the internal revenue code to: (1) increase the allowable amount of the tax credit for retirement savings; (2) expand eligibility for such credit to taxpayers with adjusted gross incomes up to $95,000 ($150,000 for joint returns); (3) establish tax-exempt roth individual retirement accounts (iras) for individuals under the age of 25; and (4) allow a tax credit for contributions to such iras.'

In [17]:
EXAMPLE_TEXT_INDEX = 0
example_text = cleaned_billsum_ds["test"]["summary"][EXAMPLE_TEXT_INDEX]
example_text

'pharmacy education aid act of 2003 - amends the public health service act to permit payments of up to $35,000 on behalf of a qualifying individual with a pharmacy degree for the repayment of pharmacy education loans for each year (two-year minimum) that such person serves in a health care facility with a critical shortage of pharmacists. provides for: (1) additional tax liability payments; (2) financial need preference; and (3) federal repayment for recipient breach of agreement unless otherwise waived.directs the secretary to make payments of up to $35,000 on behalf of a qualifying individual with a pharmacy degree or in the final year of such study for the repayment of pharmacy educational loans for each year (two-year minimum) that such person serves as a faculty member at a school of pharmacy which provides assistance to: (1) medically underserved areas; (2) prisons; (3) veterans or the armed forces; (4) the indian health service; or (5) a disproportionate share hospital under the

In [18]:
MODEL_NAME= "google/pegasus-large"

In [19]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 256, but your input_length is only 190. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)


[{'summary_text': 'provides for: (1) additional tax liability payments; (2) financial need preference; and (3) federal repayment for recipient breach of agreement unless otherwise waived.directs the secretary to make payments of up to $35,000 on behalf of a qualifying individual with a pharmacy degree or in the final year of such study for the repayment of pharmacy educational loans for each year (two-year minimum) that such person serves as a faculty member at a school of pharmacy which provides assistance to: (1) medically underserved areas; (2) prisons; (3) veterans or the armed forces; (4) the indian health service; or (5) a disproportionate share hospital under the social security act .'}]

In [20]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)
summary_txt = summarizer(prefix + example_text)
summary_txt

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 256, but your input_length is only 190. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)


[{'summary_text': 'provides for: (1) additional tax liability payments; (2) financial need preference; and (3) federal repayment for recipient breach of agreement unless otherwise waived.directs the secretary to make payments of up to $35,000 on behalf of a qualifying individual with a pharmacy degree or in the final year of such study for the repayment of pharmacy educational loans for each year (two-year minimum) that such person serves as a faculty member at a school of pharmacy which provides assistance to: (1) medically underserved areas; (2) prisons; (3) veterans or the armed forces; (4) the indian health service; or (5) a disproportionate share hospital under the social security act .'}]

In [21]:
ref_txt = cleaned_billsum_ds["test"]["title"][EXAMPLE_TEXT_INDEX]
ref_txt

'to amend the public health service act with respect to health professions programs regarding the practice of pharmacy.'

In [22]:
import evaluate
rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [23]:
reference = "Albrechts karriere ble definert av hans store engasjement for sjeldent fremført repertoar, noe som ga ham et rykte for å gjenopplive skjulte musikalske perler."

ex_1 = "Albrechts dedikasjon til å avdekke skjulte musikalske perler var tydelig i hans store engasjement for sjeldent fremført repertoar, og brakte glemte stykker til live på scenen."

ex_2 = "Hans lidenskap for mangfold i klassisk musikkprogrammering førte til at Albrecht fokuserte på sjeldent fremført repertoar, og ga publikum en unik lytteopplevelse."


In [24]:
results = rouge.compute(predictions = [ex_1], references = [reference])
results

{'rouge1': 0.48000000000000004,
 'rouge2': 0.37499999999999994,
 'rougeL': 0.35999999999999993,
 'rougeLsum': 0.35999999999999993}

In [25]:
results = rouge.compute(predictions = [ex_2], references = [reference])
results

{'rouge1': 0.2916666666666667,
 'rouge2': 0.13043478260869565,
 'rougeL': 0.2916666666666667,
 'rougeLsum': 0.2916666666666667}

In [26]:
summary_txt[0]["summary_text"]

'provides for: (1) additional tax liability payments; (2) financial need preference; and (3) federal repayment for recipient breach of agreement unless otherwise waived.directs the secretary to make payments of up to $35,000 on behalf of a qualifying individual with a pharmacy degree or in the final year of such study for the repayment of pharmacy educational loans for each year (two-year minimum) that such person serves as a faculty member at a school of pharmacy which provides assistance to: (1) medically underserved areas; (2) prisons; (3) veterans or the armed forces; (4) the indian health service; or (5) a disproportionate share hospital under the social security act .'

In [27]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.15625,
 'rouge2': 0.031746031746031744,
 'rougeL': 0.078125,
 'rougeLsum': 0.078125}

In [28]:
article_texts = cleaned_billsum_ds["test"]["title"]

article_categories = cleaned_billsum_ds["test"]["summary"]

In [29]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [08:25<00:00, 10.11s/it]


In [30]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_categories[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.16929415853761134,
 'rouge2': 0.09523522478063774,
 'rougeL': 0.13887421250696938,
 'rougeLsum': 0.13834521632308766}

In [31]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_categories[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.13903743315508021,
  0.1869688385269122,
  0.20289855072463767,
  0.2553191489361702,
  0.05128205128205129,
  0.07326007326007326,
  0.2280701754385965,
  0.2731277533039647,
  0.4413793103448276,
  0.11965811965811966,
  0.3205128205128205,
  0.027777777777777776,
  0.24242424242424243,
  0.10144927536231883,
  0.051948051948051945,
  0.14814814814814814,
  0.03257328990228013,
  0.15999999999999998,
  0.11382113821138212,
  0.028776978417266192,
  0.25,
  0.08465608465608467,
  0.06289308176100629,
  0.12955465587044532,
  0.07741935483870968,
  0.05063291139240506,
  0.38554216867469876,
  0.08130081300813008,
  0.6222222222222221,
  0.007677543186180422,
  0.13688212927756654,
  0.1581920903954802,
  0.4380952380952381,
  0.31746031746031744,
  0.25,
  0.037037037037037035,
  0.03265306122448979,
  0.303030303030303,
  0.057971014492753624,
  0.36999999999999994,
  0.09326424870466321,
  0.014705882352941175,
  0.07518796992481203,
  0.2845188284518828,
  0.038554216

In [32]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(28, 24)

In [33]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_categories[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,summarize: to amend the public health service ...,pharmacy education aid act of 2003 - amends th...
1,summarize: to amend the cheyenne river sioux t...,cheyenne river sioux tribe equitable compensat...
2,summarize: to prohibit the obtaining of custom...,consumer telephone records protection act of 2...
3,summarize: to permit certain funds assessed fo...,enron employee pension recovery act of 2002 - ...
4,summarize: colusa basin watershed integrated r...,colusa basin watershed integrated resources ma...


In [34]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(20, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(20, "Reference_summaries"))

Actual Summary
summarize: to amend the balanced budget and emergency deficit control act of 1985 and the congressional budget act of 1974 to extend the discretionary spending caps and the pay-as-you-go requirement, and for other purposes.
Reference Summary
assuring honesty and accountability act of 2002 - amends the balanced budget and emergency deficit control act of 1985 to extend through fy 2007 the discretionary spending limits (spending caps) for defense and nondefense categories in new budget authority and outlays.retains (eliminates the expiration of) certain budget enforcement mechanisms.retains (eliminates the expiration of) pay-as-you-go.amends the congressional budget act of 1974 concerning legislation which: (1) evades specified budget enforcement mechanisms; (2) provides direct spending (to be included in the congressional budget office (cbo) analysis (scoring); and (3) is unreported by committee (for purposes of budget point of order rules).limits the adjustment required 

In [35]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(18, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(18, "Reference_summaries"))

Actual Summary
summarize: rhinoceros and tiger conservation act of 1994
Reference Summary
rhinoceros and tiger conservation act of 1994 - requires the secretary of the interior to provide financial assistance for conservation of rhinoceros and tiger projects. authorizes the convention on international trade in endangered species of wild fauna and flora (cites) secretariat, or any other person, to submit to the secretary a project proposal for the conservation of such animals.  provides for approval criteria and project reporting.  directs the secretary to consider projects which will enhance sustainable development programs to ensure effective, long-term conservation of rhinoceros and tigers. (sec. 6) establishes in the treasury the rhinoceros and tiger conservation fund.  provides for fund deposits, fund use, and acceptance and use of donations. (sec. 7) authorizes appropriations.


In [36]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [37]:
tokenizer(["four-time defending champion"])

{'input_ids': [[541, 121, 1139, 10838, 6232, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [38]:
tokens = tokenizer.tokenize("four-time defending champion")
tokens


['▁four', '-', 'time', '▁defending', '▁champion']

In [39]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[541, 121, 1139, 10838, 6232]

In [40]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length = 512, truncation = True)

    labels = tokenizer(text_target = examples["title"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [44]:
tokenized_billsum_ds = cleaned_billsum_ds.map(preprocess_function, batched = True)

tokenized_billsum_ds

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 454
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 114
    })
})

In [45]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [46]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [47]:
from huggingface_hub import notebook_login
notebook_login()

In [51]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "Fine-Tuned-billsum-Summarization",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_billsum_ds["train"],
    eval_dataset = tokenized_billsum_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 174.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 133.06 MiB is free. Process 2468 has 14.62 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 95.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.push_to_hub()